In [1]:
import requests
from pathlib import Path
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib3
import requests_cache
from tqdm import tqdm
import langid
import numpy as np
import openai
from openai import OpenAI
import json
import hashlib

In [2]:
requests_cache.install_cache('.cache') 

In [3]:
# 🔇 Disable SSL warnings (for local, trusted scraping only)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [13]:
DATA = Path('~/Downloads')
# Cache directory inside Downloads
CACHE_DIR = DATA / "chatgpt_cache"
CACHE_DIR.mkdir(parents=True, exist_ok=True)
print(f"✅ Cache directory set to: {CACHE_DIR.resolve()}")

✅ Cache directory set to: /Users/qiang/Documents/mali_trait/~/Downloads/chatgpt_cache


In [8]:
wfo_df = pd.read_csv(DATA / 'classification.csv', delimiter="\t", encoding="latin1")

In [12]:
file1 = '~/Downloads/WomenPlantGenera_v1-3.csv'
data = pd.read_csv(file1,chunksize=10000,encoding='utf-8',on_bad_lines='skip',engine='python')
data = pd.concat(data)
columns = data.columns.tolist()

columns = data.columns.tolist()   ## load content into list
# data.head()

In [6]:
wfo_df.head()

,taxonID,scientificNameID,localID,scientificName,taxonRank,parentNameUsageID,scientificNameAuthorship,family,subfamily,tribe,...,acceptedNameUsageID,originalNameUsageID,nameAccordingToID,taxonRemarks,created,modified,references,source,majorGroup,tplID
0,wfo-0001302010,NaN,9905237,Schoenoxiphium ecklonii var. ecklonii,variety,NaN,NaN,Cyperaceae,NaN,NaN,...,NaN,NaN,NaN,Source in seed data: tro More details could be...,2022-04-16,2023-05-29,NaN,The Cyperaceae TEN,A,tro-9905237
1,wfo-0001302011,NaN,9905253,Cyperus violifolia,species,NaN,Rodriguez & Alfonso,Cyperaceae,NaN,NaN,...,NaN,NaN,NaN,Source in seed data: tro More details could be...,2022-04-16,2022-04-20,NaN,The Cyperaceae TEN,A,tro-9905253
2,wfo-0001302012,NaN,9905297,Carex viridula var. viridula,variety,NaN,NaN,Cyperaceae,NaN,NaN,...,NaN,NaN,NaN,Source in seed data: tro More details could be...,2022-04-16,2023-05-29,NaN,The Cyperaceae TEN,A,tro-9905297
3,wfo-0001302013,urn:lsid:ipni.org:names:310980-1,9905332,Mariscus phleoides,species,NaN,Nees,Cyperaceae,NaN,NaN,...,NaN,NaN,NaN,Source in seed data: tro Updated namePublished...,2022-04-16,2024-06-04,NaN,The Cyperaceae TEN,A,tro-9905332
4,wfo-0001302014,urn:lsid:ipni.org:names:315251-1,9905359,Tetraria compar,species,NaN,P.Beauv. ex T.Lestib.,Cyperaceae,NaN,NaN,...,NaN,NaN,NaN,Source in seed data: tro Updated Author from L...,2022-04-16,2024-06-04,NaN,The Cyperaceae TEN,A,tro-9905359


In [7]:
gbif_df = pd.read_csv(DATA / 'gbif-mali-taxonomy.csv')

In [8]:
gbif_df.species

0       Phalaris arundinacea
1         Pandanus malgrasii
2       Platycoryne paludosa
3        Nervilia bicarinata
4           Nervilia petraea
                ...         
1531                     NaN
1532        Coccinia grandis
1533                     NaN
1534      Archidium ohioense
1535                     NaN
Name: species, Length: 1536, dtype: object

In [9]:
mali_wfo_df = wfo_df[wfo_df.scientificName.isin(gbif_df.species)]

In [10]:
def scrape_wfo_paragraphs(wfo_taxon_id):
    url = f"https://www.worldfloraonline.org/taxon/{wfo_taxon_id}"
    
    try:
        response = requests.get(url, verify=False, timeout=10)  # disable cert check
    except requests.exceptions.SSLError as e:
        print(f"SSL Error for {wfo_taxon_id}: {e}")
        return []
    except Exception as e:
        print(f"Request error for {wfo_taxon_id}: {e}")
        return []

    if response.status_code != 200:
        print(f"❌ Failed to retrieve page for {wfo_taxon_id}")
        return []

    soup = BeautifulSoup(response.content, "html.parser")
    section_ids = ['general', 'morphology', 'habit', 'ecology', 'habitat', 'distribution', 'use']

    records = []
    for sec_id in section_ids:
        section_div = soup.find("div", id=sec_id)
        if section_div:
            desc_blocks = section_div.find_all("div", class_="description-with-citations")
            for block in desc_blocks:
                paragraphs = block.find_all("p", class_="justified")
                for para in paragraphs:
                    records.append({
                        "wfo_taxon_id": wfo_taxon_id,
                        "section": sec_id,
                        "text": para.get_text(strip=True)
                    })
    return records

In [2]:
file = '~/Downloads/trait/mali_wfo_df.csv'
mali_wfo_df = pd.read_csv(file, encoding="latin1")  # no delimiter specified
print(len(mali_wfo_df))

1169


In [12]:

results = []

for i, row in tqdm(mali_wfo_df.iterrows(), total=len(mali_wfo_df), desc="Processing WFO taxa"):
    name = row['scientificName']
    taxonID = row['taxonID']    
    result = scrape_wfo_paragraphs(taxonID)
    result = [{**d, 'name': name} for d in result]
    results.extend(result)
    time.sleep(3)  # 🔑 throttle: adjust if needed
    

Processing WFO taxa:   9%|██▊                             | 105/1169 [05:23<54:48,  3.09s/it]

SSL Error for wfo-0000177471: HTTPSConnectionPool(host='www.worldfloraonline.org', port=443): Max retries exceeded with url: /taxon/wfo-0000177471 (Caused by SSLError(SSLZeroReturnError(6, 'TLS/SSL connection has been closed (EOF) (_ssl.c:1133)')))


Processing WFO taxa:  16%|█████▏                          | 190/1169 [09:44<49:38,  3.04s/it]

SSL Error for wfo-0000198957: HTTPSConnectionPool(host='www.worldfloraonline.org', port=443): Max retries exceeded with url: /taxon/wfo-0000198957 (Caused by SSLError(SSLZeroReturnError(6, 'TLS/SSL connection has been closed (EOF) (_ssl.c:1133)')))


Processing WFO taxa:  29%|█████████▏                      | 337/1169 [17:25<57:27,  4.14s/it]

...

Processing WFO taxa: 100%|████████████████████████████▉| 1167/1169 [1:05:43<00:07,  3.73s/it]

SSL Error for wfo-1000074621: HTTPSConnectionPool(host='www.worldfloraonline.org', port=443): Max retries exceeded with url: /taxon/wfo-1000074621 (Caused by SSLError(SSLZeroReturnError(6, 'TLS/SSL connection has been closed (EOF) (_ssl.c:1133)')))


Processing WFO taxa: 100%|████████████████████████████▉| 1168/1169 [1:05:47<00:03,  3.61s/it]

SSL Error for wfo-1200107592: HTTPSConnectionPool(host='www.worldfloraonline.org', port=443): Max retries exceeded with url: /taxon/wfo-1200107592 (Caused by SSLError(SSLZeroReturnError(6, 'TLS/SSL connection has been closed (EOF) (_ssl.c:1133)')))


Processing WFO taxa: 100%|█████████████████████████████| 1169/1169 [1:05:50<00:00,  3.38s/it]


In [13]:
results_df = pd.DataFrame(results)
len(results_df)

2202

In [16]:
def scrape_wfo_paragraphs(wfo_taxon_id):
    url = f"https://www.worldfloraonline.org/taxon/{wfo_taxon_id}"
    
    try:
        session = requests.Session()
        response = session.get(url, verify=False, timeout=15)
        response.raise_for_status()
        # response = requests.get(url, verify=False, timeout=10)  # disable cert check
    except requests.exceptions.SSLError as e:
        print(f"SSL Error for {wfo_taxon_id}: {e}")
        return []
    except Exception as e:
        print(f"Request error for {wfo_taxon_id}: {e}")
        return []

    if response.status_code != 200:
        print(f"❌ Failed to retrieve page for {wfo_taxon_id}")
        return []

    soup = BeautifulSoup(response.content, "html.parser")
    section_ids = ['general', 'morphology', 'habit', 'ecology', 'habitat', 'distribution', 'use']

    records = []
    for sec_id in section_ids:
        section_div = soup.find("div", id=sec_id)
        if section_div:
            desc_blocks = section_div.find_all("div", class_="description-with-citations")
            for block in desc_blocks:
                paragraphs = block.find_all("p", class_="justified")
                for para in paragraphs:
                    records.append({
                        "wfo_taxon_id": wfo_taxon_id,
                        "section": sec_id,
                        "text": para.get_text(strip=True)
                    })
    return records

In [35]:
# Load failed IDs
with open("failed_ids_remaining_3.txt", "r") as f:
    failed_ids = [line.strip() for line in f if line.strip()]

print(f"🔄 Retrying {len(failed_ids)} failed IDs...")

🔄 Retrying 419 failed IDs...


In [29]:
# Build a lookup dictionary: taxonID → scientificName
id_to_name = dict(zip(mali_wfo_df["taxonID"], mali_wfo_df["scientificName"]))

In [36]:
still_failed_all = []
count = 1

# Process chunks sequentially
for chunk_idx in range(0, len(failed_ids), 200):
    chunk = failed_ids[chunk_idx:chunk_idx+200]
    retry_results = []
    still_failed = []

    for taxonID in tqdm(chunk, desc=f"Retrying chunk {chunk_idx//200+1}"):
        name = id_to_name.get(taxonID, "UNKNOWN")
        result = scrape_wfo_paragraphs(taxonID)
        if not result:
            still_failed.append(taxonID)
        else:
            result = [{**d, "name": name} for d in result]
            retry_results.extend(result)
        time.sleep(5)  # throttle

    # Save this chunk’s results
    if retry_results:
        out_file = f"retry_results_chunk{count}.csv"
        pd.DataFrame(retry_results).to_csv(out_file, index=False)
        print(f"✅ Saved {len(retry_results)} rows to {out_file}")

    # Collect still failing IDs
    still_failed_all.extend(still_failed)
    count = count +1

# Save all still-failing IDs after processing all chunks
if still_failed_all:
    new_failed_file = "failed_ids_remaining_4.txt"
    with open(new_failed_file, "w") as f:
        f.write("\n".join(still_failed_all))
    print(f"⚠️ {len(still_failed_all)} IDs still failing → saved to {new_failed_file}")
else:
    print("🎉 All failed IDs processed successfully")

Retrying chunk 2: 100%|████████████████████████████████████| 200/200 [19:31<00:00,  5.86s/it]


✅ Saved 314 rows to retry_results_chunk2.csv


Retrying chunk 3: 100%|██████████████████████████████████████| 19/19 [01:35<00:00,  5.04s/it]

⚠️ 368 IDs still failing → saved to failed_ids_remaining_4.txt


In [23]:
count =1
out_file = f"retry_results_chunk_{count}.csv"
print(out_file)

retry_results_chunk_1.csv


In [37]:
len(still_failed_all)

368

In [32]:
if still_failed_all:
    new_failed_file = "failed_ids_remaining_2.txt"
    with open(new_failed_file, "w") as f:
        f.write("\n".join(still_failed_all))
    print(f"⚠️ {len(still_failed_all)} IDs still failing → saved to {new_failed_file}")
else:
    print("🎉 All failed IDs processed successfully")

⚠️ 419 IDs still failing → saved to failed_ids_remaining_2.txt


In [38]:
base_dir = Path("scraped_data")

# Start with the main results file
merged_df = pd.read_csv(base_dir / "results_0.csv")

# Loop over retry_* directories and read all CSVs inside
for retry_dir in base_dir.glob("retry_*"):
    for csv_file in retry_dir.glob("*.csv"):
        print(f"Adding {csv_file}")
        df = pd.read_csv(csv_file)
        merged_df = pd.concat([merged_df, df], ignore_index=True)

# Save final merged results
output_file = base_dir / "merged_results.csv"
merged_df.to_csv(output_file, index=False)
print(f"✅ Merged file saved to {output_file} with {len(merged_df)} rows")


Adding scraped_data/retry_3/retry_results_chunk1.csv
Adding scraped_data/retry_3/retry_results_chunk3.csv
Adding scraped_data/retry_3/retry_results_chunk2.csv
Adding scraped_data/retry_4/retry_results_chunk2.csv
Adding scraped_data/retry_2/retry_results_chunk1.csv
Adding scraped_data/retry_0/retry_results_chunk1.csv
Adding scraped_data/retry_1/retry_results_chunk1.csv
Adding scraped_data/retry_1/retry_results_chunk3.csv
Adding scraped_data/retry_1/retry_results_chunk2.csv
✅ Merged file saved to scraped_data/merged_results.csv with 5450 rows


In [40]:
merged_df

,wfo_taxon_id,section,text,name
0,wfo-0001302265,general,"Annuals. Roots many, fibrous. Culms densely tu...",Cyperus michelianus
1,wfo-0001302265,general,"Tufted annual, 3-10 cm. Roots fibrous. Stem tr...",Cyperus michelianus
2,wfo-0001302265,general,"Annuals. Roots many, fibrous. Culms densely tu...",Cyperus michelianus
3,wfo-0001302265,general,"Tufted annual, 3-10 cm. Roots fibrous. Stem tr...",Cyperus michelianus
4,wfo-0000002780,morphology,"An annual herb 1-3 ft. high with glabrous, whi...",Sphaeranthus talbotii
...,...,...,...,...
5445,wfo-0000531553,morphology,Climber or liane with cylindric hollow glabrou...,Ampelocissus multistriata
5446,wfo-0000531553,morphology,"Calyx 1 mm. in diam., salver-shaped, glabrous,...",Ampelocissus multistriata
5447,wfo-0000531553,morphology,"Liane'à tiges rondes, fermes, striées, de ± 13...",Ampelocissus multistriata
5448,wfo-0000531553,ecology,In the savannah regions.,Ampelocissus multistriata


In [41]:
output_file = base_dir / "merged_results_dedup.csv"
print(f"Before deduplication: {len(merged_df)} rows")

# Drop exact duplicate rows (all columns must match)
df_dedup = merged_df.drop_duplicates()

print(f"After deduplication: {len(df_dedup)} rows")

# Save deduplicated dataset
df_dedup.to_csv(output_file, index=False)
print(f"✅ Deduplicated file saved to {output_file}")

Before deduplication: 5450 rows
After deduplication: 5270 rows
✅ Deduplicated file saved to scraped_data/merged_results_dedup.csv


In [42]:
df_dedup

,wfo_taxon_id,section,text,name
0,wfo-0001302265,general,"Annuals. Roots many, fibrous. Culms densely tu...",Cyperus michelianus
1,wfo-0001302265,general,"Tufted annual, 3-10 cm. Roots fibrous. Stem tr...",Cyperus michelianus
4,wfo-0000002780,morphology,"An annual herb 1-3 ft. high with glabrous, whi...",Sphaeranthus talbotii
5,wfo-0000003146,general,"Por Michael O. Dillon, emend. J. Pruski, 2003M...",Matricaria chamomilla
6,wfo-0000009712,morphology,"Perennial or annual herb, decumbent or scrambl...",Acmella caulirhiza
...,...,...,...,...
5445,wfo-0000531553,morphology,Climber or liane with cylindric hollow glabrou...,Ampelocissus multistriata
5446,wfo-0000531553,morphology,"Calyx 1 mm. in diam., salver-shaped, glabrous,...",Ampelocissus multistriata
5447,wfo-0000531553,morphology,"Liane'à tiges rondes, fermes, striées, de ± 13...",Ampelocissus multistriata
5448,wfo-0000531553,ecology,In the savannah regions.,Ampelocissus multistriata


In [12]:
results_df = pd.DataFrame(results)

In [13]:
results_df

,wfo_taxon_id,section,text,name
0,wfo-0001302265,general,"Annuals. Roots many, fibrous. Culms densely tu...",Cyperus michelianus
1,wfo-0001302265,general,"Tufted annual, 3-10 cm. Roots fibrous. Stem tr...",Cyperus michelianus
2,wfo-0000002780,morphology,"An annual herb 1-3 ft. high with glabrous, whi...",Sphaeranthus talbotii
3,wfo-0000003146,general,"Por Michael O. Dillon, emend. J. Pruski, 2003M...",Matricaria chamomilla
4,wfo-0000009712,morphology,"Perennial or annual herb, decumbent or scrambl...",Acmella caulirhiza
...,...,...,...,...
5293,wfo-0001425157,habitat,Pans. It occurs along the sides of shallow sea...,Pontederia africana
5294,wfo-0001425157,distribution,The only known record for the region is from T...,Pontederia africana
5295,wfo-1000031633,morphology,"Herbeannuelle, de 10–100 cm de haut, aromatiqu...",Coleus rhodesianum
5296,wfo-1000031633,ecology,pionnière des sols perturbés ; au Gabon à 100–...,Coleus rhodesianum


In [43]:
def detect_langid(text):
    # Get top two language predictions and scores
    ranked = langid.rank(text)
    top_lang, top_score = ranked[0]
    second_lang, second_score = ranked[1]

    # Convert to softmax percentage confidence
    scores = np.array([top_score, second_score])
    probs = np.exp(scores - np.max(scores))  # for numerical stability
    probs = probs / probs.sum()

    return top_lang, float(probs[0]) * 100  # return as percentage

In [44]:
def apply_detect_langid(text):
    lang, conf = detect_langid(text)
    return pd.Series({'lang': lang, 'lang_confidence': conf})

In [45]:
df_dedup[['lang', 'lang_confidence']] = df_dedup['text'].apply(apply_detect_langid)

/var/folders/gf/wqsz1zkd6x95xx3fgy6qr2780000gn/T/ipykernel_74293/1451674328.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dedup[['lang', 'lang_confidence']] = df_dedup['text'].apply(apply_detect_langid)
/var/folders/gf/wqsz1zkd6x95xx3fgy6qr2780000gn/T/ipykernel_74293/1451674328.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dedup[['lang', 'lang_confidence']] = df_dedup['text'].apply(apply_detect_langid)


In [46]:
df_dedup

,wfo_taxon_id,section,text,name,lang,lang_confidence
0,wfo-0001302265,general,"Annuals. Roots many, fibrous. Culms densely tu...",Cyperus michelianus,en,100.000000
1,wfo-0001302265,general,"Tufted annual, 3-10 cm. Roots fibrous. Stem tr...",Cyperus michelianus,en,100.000000
4,wfo-0000002780,morphology,"An annual herb 1-3 ft. high with glabrous, whi...",Sphaeranthus talbotii,en,100.000000
5,wfo-0000003146,general,"Por Michael O. Dillon, emend. J. Pruski, 2003M...",Matricaria chamomilla,es,100.000000
6,wfo-0000009712,morphology,"Perennial or annual herb, decumbent or scrambl...",Acmella caulirhiza,en,100.000000
...,...,...,...,...,...,...
5445,wfo-0000531553,morphology,Climber or liane with cylindric hollow glabrou...,Ampelocissus multistriata,en,100.000000
5446,wfo-0000531553,morphology,"Calyx 1 mm. in diam., salver-shaped, glabrous,...",Ampelocissus multistriata,en,100.000000
5447,wfo-0000531553,morphology,"Liane'à tiges rondes, fermes, striées, de ± 13...",Ampelocissus multistriata,fr,100.000000
5448,wfo-0000531553,ecology,In the savannah regions.,Ampelocissus multistriata,en,99.999989


In [47]:
output_file = base_dir / "merged_results_dedup_lang.csv"
df_dedup.to_csv(output_file, index=False)

In [ ]:
results_df[['lang', 'lang_confidence']] = results_df['text'].apply(apply_detect_langid)

In [16]:
results_df

,wfo_taxon_id,section,text,name,lang,lang_confidence
0,wfo-0001302265,general,"Annuals. Roots many, fibrous. Culms densely tu...",Cyperus michelianus,en,100.0
1,wfo-0001302265,general,"Tufted annual, 3-10 cm. Roots fibrous. Stem tr...",Cyperus michelianus,en,100.0
2,wfo-0000002780,morphology,"An annual herb 1-3 ft. high with glabrous, whi...",Sphaeranthus talbotii,en,100.0
3,wfo-0000003146,general,"Por Michael O. Dillon, emend. J. Pruski, 2003M...",Matricaria chamomilla,es,100.0
4,wfo-0000009712,morphology,"Perennial or annual herb, decumbent or scrambl...",Acmella caulirhiza,en,100.0
...,...,...,...,...,...,...
5293,wfo-0001425157,habitat,Pans. It occurs along the sides of shallow sea...,Pontederia africana,en,100.0
5294,wfo-0001425157,distribution,The only known record for the region is from T...,Pontederia africana,en,100.0
5295,wfo-1000031633,morphology,"Herbeannuelle, de 10–100 cm de haut, aromatiqu...",Coleus rhodesianum,fr,100.0
5296,wfo-1000031633,ecology,pionnière des sols perturbés ; au Gabon à 100–...,Coleus rhodesianum,fr,100.0


In [48]:
desc_df = df_dedup[(df_dedup.section.isin(['general', 'morphology'])) & (df_dedup.lang.isin(['fr', 'en']))]

In [49]:
desc_df

,wfo_taxon_id,section,text,name,lang,lang_confidence
0,wfo-0001302265,general,"Annuals. Roots many, fibrous. Culms densely tu...",Cyperus michelianus,en,100.0
1,wfo-0001302265,general,"Tufted annual, 3-10 cm. Roots fibrous. Stem tr...",Cyperus michelianus,en,100.0
4,wfo-0000002780,morphology,"An annual herb 1-3 ft. high with glabrous, whi...",Sphaeranthus talbotii,en,100.0
6,wfo-0000009712,morphology,"Perennial or annual herb, decumbent or scrambl...",Acmella caulirhiza,en,100.0
7,wfo-0000009712,morphology,"Herbeannuelle ou pérenne, décombante ou presqu...",Acmella caulirhiza,fr,100.0
...,...,...,...,...,...,...
5443,wfo-0000531536,morphology,Pubescent and with purple gland-tipped setae F...,Ampelocissus leonensis,en,100.0
5444,wfo-0000531553,morphology,"Fruits smooth, grape-like Inflorescence crowde...",Ampelocissus multistriata,en,100.0
5445,wfo-0000531553,morphology,Climber or liane with cylindric hollow glabrou...,Ampelocissus multistriata,en,100.0
5446,wfo-0000531553,morphology,"Calyx 1 mm. in diam., salver-shaped, glabrous,...",Ampelocissus multistriata,en,100.0


In [50]:
output_file = base_dir / "desc_df.csv"
desc_df.to_csv(output_file, index=False)

In [3]:
processed_df = pd.read_csv('wfo-traits-enfr_1.csv', encoding="latin1")

In [4]:
processed_df.wfo_taxon_id

0      wfo-0001302265
1      wfo-0001302265
2      wfo-0000002780
3      wfo-0000009712
4      wfo-0000009712
            ...      
930    wfo-0000394468
931    wfo-0000394468
932    wfo-0000394468
933    wfo-0000394468
934    wfo-0000394468
Name: wfo_taxon_id, Length: 935, dtype: object

In [5]:
processed_df.wfo_taxon_id.value_counts()

wfo_taxon_id
wfo-0000219580    10
wfo-0000371248    10
wfo-0000179216    10
wfo-0000188562     9
wfo-0000165078     9
                  ..
wfo-0000342307     1
wfo-0000002780     1
wfo-0000170254     1
wfo-0000116880     1
wfo-0000366394     1
Name: count, Length: 305, dtype: int64

In [6]:
processed2_df = pd.read_csv('wfo-traits-enfr-2.csv')
processed2_df.wfo_taxon_id.value_counts()

wfo_taxon_id
wfo-0000484425    13
wfo-0000457857    12
wfo-0000516177    12
wfo-0000547034    11
wfo-0000587875    11
                  ..
wfo-0000653925     1
wfo-0001299065     1
wfo-0001336544     1
wfo-0001425157     1
wfo-0000541028     1
Name: count, Length: 488, dtype: int64

In [56]:
unprocessed_df = desc_df[~desc_df["wfo_taxon_id"].isin(processed_df["wfo_taxon_id"])]
print(f"Unprocessed records: {len(unprocessed_df)} rows")

Unprocessed records: 1537 rows


In [58]:
output_file = base_dir / "merged_results_unprocessed.csv"
unprocessed_df.to_csv(output_file, index=False)
print(f"✅ Unprocessed file saved to {output_file}")

✅ Unprocessed file saved to scraped_data/merged_results_unprocessed.csv


## Extract traits

In [16]:
OPENAI_MODEL = "gpt-4"  # or "gpt-3.5-turbo"
OPENAI_API_KEY = "sk-proj-"



In [17]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [18]:
system_message = """You are an expert in botanical trait extraction.

Your task is to extract ONLY the specified trait fields from a botanical species description. Do not guess or infer values — only extract what is explicitly stated. If a trait is not mentioned, set its value to null.

Return your output strictly as a JSON object with exactly the keys listed. Do not add any explanations or extra text.
"""

prompt_fr = """
Extract the following trait fields from the botanical description below. If a field is not mentioned, set it to null.

Trait fields:
"life form", "habitat", "habit", "clonality", "perennial organ", "plant min. height [m]",
"plant max. height [m]", "indumentum", "spinescence", "succulence", "leaf arrangement",
"leaf architecture", "leaf position", "leaf shape", "leaf apex", "leaf base", "leaf margin",
"leaf min. width [cm]", "leaf max. width [cm]", "leaf min. length [cm]", "leaf max. length [cm]",
"bract colour", "bracteole colour", "inflorescence arrangement", "inflorescence orientation",
"flower sex", "flower architecture", "flower merosity", "flower symmetry", "flower shape",
"flower colour", "flower orientation", "petal fusion", "petal colour", "petal min. width [cm]",
"petal max. width [cm]", "calyx colour", "calyx min. length [cm]", "calyx max. length [cm]",
"corolla colour", "corolla tube min. length [cm]", "corolla tube max. length [cm]",
"corolla tube lipped", "labellum colour", "labellum min. width [cm]", "labellum max. width [cm]",
"spur min. length [cm]", "spur max. length [cm]", "reproduction architecture", "reproduction system",
"stamen number", "stamen arrangement", "stamenoid number", "stamen min. length [cm]",
"stamen max. length [cm]", "staminodium", "pollen colour", "carpel/ovary number",
"gynoecium arrangement", "heterostyly", "pollination", "reward", "fruit type", "fruit structure",
"fruit dehiscence", "fruit shape", "fruit colour", "seed colour", "seeds per fruit min",
"seeds per fruit max", "seed min. width [mm]", "seed max. width [mm]", "seed min. length [mm]",
"seed max. length [mm]", "seed min. diameter [mm]", "seed max. diameter [mm]", "dispersal mode",
"ploidy level (2n)", "root min. depth [cm]", "root max. depth [cm]"

Botanical description:
<<<
{description}
>>>

You will be given a French botanical description. Translate the French technical terms faithfully into English. 

"""

prompt_en = """
Extract the following trait fields from the botanical description below. If a field is not mentioned, set it to null.

Trait fields:
"life form", "habitat", "habit", "clonality", "perennial organ", "plant min. height [m]",
"plant max. height [m]", "indumentum", "spinescence", "succulence", "leaf arrangement",
"leaf architecture", "leaf position", "leaf shape", "leaf apex", "leaf base", "leaf margin",
"leaf min. width [cm]", "leaf max. width [cm]", "leaf min. length [cm]", "leaf max. length [cm]",
"bract colour", "bracteole colour", "inflorescence arrangement", "inflorescence orientation",
"flower sex", "flower architecture", "flower merosity", "flower symmetry", "flower shape",
"flower colour", "flower orientation", "petal fusion", "petal colour", "petal min. width [cm]",
"petal max. width [cm]", "calyx colour", "calyx min. length [cm]", "calyx max. length [cm]",
"corolla colour", "corolla tube min. length [cm]", "corolla tube max. length [cm]",
"corolla tube lipped", "labellum colour", "labellum min. width [cm]", "labellum max. width [cm]",
"spur min. length [cm]", "spur max. length [cm]", "reproduction architecture", "reproduction system",
"stamen number", "stamen arrangement", "stamenoid number", "stamen min. length [cm]",
"stamen max. length [cm]", "staminodium", "pollen colour", "carpel/ovary number",
"gynoecium arrangement", "heterostyly", "pollination", "reward", "fruit type", "fruit structure",
"fruit dehiscence", "fruit shape", "fruit colour", "seed colour", "seeds per fruit min",
"seeds per fruit max", "seed min. width [mm]", "seed max. width [mm]", "seed min. length [mm]",
"seed max. length [mm]", "seed min. diameter [mm]", "seed max. diameter [mm]", "dispersal mode",
"ploidy level (2n)", "root min. depth [cm]", "root max. depth [cm]"

Botanical description:
<<<
{description}
>>>

"""

def get_cache_key(prompt, model, temperature):
    data = {
        "prompt": prompt,
        "model": model,
        "temperature": temperature
    }
    data_string = json.dumps(data, sort_keys=True)
    return hashlib.sha256(data_string.encode()).hexdigest()

def call_chatgpt_with_cache(prompt, model="gpt-4-1106-preview", temperature=0):

    cache_key = get_cache_key(prompt, model, temperature)
    cache_file = CACHE_DIR / f"{cache_key}.json"

    if cache_file.exists():
        return json.loads(cache_file.read_text())
        
    response = client.chat.completions.create(
        model=model,
        # model="gpt-4o",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt.format(description=description)}
        ],
        temperature=temperature,
    )  

    content = response.choices[0].message.content

    try:
        cleaned = content.strip().strip('```json').strip('```').strip()
        data = json.loads(cleaned)
    except json.JSONDecodeError as e:
        print("Error decoding JSON:")
        print(content)
        raise e   
    else:
        cache_file.write_text(json.dumps(data))

    return data

    
    

def extract_traits(description, lang):

    prompt = prompt_en if lang == 'en' else prompt_fr
    prompt = prompt.format(description=description)

    traits = call_chatgpt_with_cache(prompt)
    return traits
    

    


# description = """
# Herbeannuelle ou pérenne, décombante ou presque dressée, souvent radicante, atteignant 15 cm de hauteur ou 60 cm de longueur ; tige aqueuse, souvent rougeâtre ou violacée, glabre ou pileuse.Feuille: pétiole 1–15 mm, étroitement ailé ; limbe ovale, 1–7 × 0,8–4 cm, base atténuée, sommet aigu ou obtus, marge dentée, glabre ou pileux.Capituleradié, solitaire, hémisphérique, devenant conique, 4–14 × 4–10 mm ; pédoncule 2–7 cm ; involucre 3,5–6 mm de longueur, àbractées bisériées.Fleurons ligulés10 à 15; corolle 2,2–3,3 mm de longueur, jaune à orange, ligule 1,4–2,3 mm de longueur,Fleurons tubulésnombreux ; corolle 1,1–1,9 mm de longueur, jaune à orange clair.Akène1,3–2,4 mm de longueur, glabre ;pappus généralement absent, mais parfois ceux des fleurons externes à quelques soies.
# """ 
# c = extract_traits(description, lang='fr')        

In [12]:
# desc_df = results_df[(results_df.section.isin(['general', 'morphology'])) & (results_df.lang.isin(['fr', 'en']))]

# for row in

In [9]:
base_dir = Path("scraped_data")
df_file = base_dir / "merged_results_unprocessed.csv"
desc_df = pd.read_csv(df_file)
print(f"Unprocessed records: {len(desc_df)} rows")

Unprocessed records: 1537 rows


In [11]:
desc_df.head()

,wfo_taxon_id,section,text,name,lang,lang_confidence
0,wfo-0000552165,general,"Annuals. Rhizomes absent. Culms tufted, erect,...",Scleria pergracilis,en,100.000000
1,wfo-0000552165,general,"Annual. Culms 20–60 cm by 0.5–1 mm, trigonous,...",Scleria pergracilis,en,100.000000
2,wfo-0000552165,morphology,Slender annual 45 cm. high; Leaves 1–2 mm. wid...,Scleria pergracilis,en,99.999935
3,wfo-0000552165,morphology,"Annual, caespitose. Culms 0.13-0.38 m tall, gl...",Scleria pergracilis,en,100.000000
4,wfo-0000553106,general,"Annual. Culms tufted, 60-90 cm by 2–3 mm, triq...",Scleria tessellata,en,100.000000


In [11]:
import time

In [20]:
CHECKPOINT_FILE = DATA / "traits_checkpoint.csv"

data = []
start_index = 0     # resume from here if restarting
save_every = 200    # save every N rows

# If checkpoint exists, resume
if CHECKPOINT_FILE.exists():
    prev = pd.read_csv(CHECKPOINT_FILE)
    data = prev.to_dict("records")
    start_index = len(prev)
    print(f"🔄 Resuming from checkpoint: {start_index} records loaded")

# Loop with enumerate for consistent checkpointing
for j, (i, row) in enumerate(
    tqdm(desc_df.iloc[start_index:].iterrows(),
         total=len(desc_df) - start_index,
         desc="Extracting traits")
):
    try:
        traits = extract_traits(row.text, row.lang)
    except Exception as e:
        print(f"⚠️ Error at global row {i}: {e}")
        # log failed rows for retry
        with open(DATA / "failed_rows.txt", "a") as f:
            f.write(f"{i},{row.wfo_taxon_id}\n")
        continue

    traits["name"] = row["name"]
    traits["lang"] = row.lang
    traits["wfo_taxon_id"] = row.wfo_taxon_id
    data.append(traits)

    # Checkpoint every N *within this run*
    if (j + 1) % save_every == 0:
        pd.DataFrame(data).to_csv(CHECKPOINT_FILE, index=False)
        print(f"💾 Checkpoint saved after {j+1} rows this run "
              f"({len(data)} total records)")
        time.sleep(1)  # be nice to API/LLM

# Final save
pd.DataFrame(data).to_csv(CHECKPOINT_FILE, index=False)
print(f"✅ Finished. Total saved: {len(data)} records")


Extracting traits: 100%|██████████████████████████████████████████████████████████████████| 124/124 [35:26<00:00, 17.15s/it]

✅ Finished. Total saved: 124 records


In [21]:
df = pd.DataFrame(data)
df

,life form,habitat,habit,clonality,perennial organ,plant min. height [m],plant max. height [m],indumentum,spinescence,succulence,...,seed max. length [mm],seed min. diameter [mm],seed max. diameter [mm],dispersal mode,ploidy level (2n),root min. depth [cm],root max. depth [cm],name,lang,wfo_taxon_id
0,None,None,verzweigt,None,None,0.3,1.3,kahl,None,None,...,NaN,None,None,None,None,None,None,Bidens bipinnata,de,wfo-0000028784
1,None,None,None,None,None,NaN,NaN,None,None,None,...,NaN,None,None,pappus 2 to 3 awned sometimes absent,None,None,None,Cosmos sulphureus,pt,wfo-0000030713
2,None,None,None,None,None,NaN,NaN,absent,None,None,...,NaN,None,None,None,None,None,None,Ambrosia artemisiifolia,pt,wfo-0000063493
3,None,None,None,None,None,0.2,0.9,abstehend behaart,None,None,...,NaN,None,None,None,None,None,None,Ambrosia artemisiifolia,de,wfo-0000063493
4,None,None,None,None,None,NaN,NaN,hispid,None,None,...,NaN,None,None,None,None,None,None,Acanthospermum hispidum,pt,wfo-0000068556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,None,None,prostrate,None,None,NaN,NaN,present,None,None,...,NaN,None,None,None,None,None,None,Alternanthera pungens,pt,wfo-0000528787
120,None,None,prostrate,None,None,NaN,NaN,absent/present,None,None,...,NaN,None,None,None,None,None,None,Alternanthera sessilis,pt,wfo-0000528825
121,None,None,None,None,None,0.3,1.0,None,None,None,...,NaN,None,None,None,None,None,None,Amaranthus caudatus,de,wfo-0000530136
122,None,None,None,None,None,NaN,NaN,glabrous,absent,None,...,NaN,None,None,None,None,None,None,Amaranthus viridis,pt,wfo-0000530552


In [22]:
col = 'name'
df = df[[col] + [c for c in df.columns if c != col]]

In [23]:
df

,name,life form,habitat,habit,clonality,perennial organ,plant min. height [m],plant max. height [m],indumentum,spinescence,...,seed min. length [mm],seed max. length [mm],seed min. diameter [mm],seed max. diameter [mm],dispersal mode,ploidy level (2n),root min. depth [cm],root max. depth [cm],lang,wfo_taxon_id
0,Bidens bipinnata,None,None,verzweigt,None,None,0.3,1.3,kahl,None,...,NaN,NaN,None,None,None,None,None,None,de,wfo-0000028784
1,Cosmos sulphureus,None,None,None,None,None,NaN,NaN,None,None,...,NaN,NaN,None,None,pappus 2 to 3 awned sometimes absent,None,None,None,pt,wfo-0000030713
2,Ambrosia artemisiifolia,None,None,None,None,None,NaN,NaN,absent,None,...,NaN,NaN,None,None,None,None,None,None,pt,wfo-0000063493
3,Ambrosia artemisiifolia,None,None,None,None,None,0.2,0.9,abstehend behaart,None,...,NaN,NaN,None,None,None,None,None,None,de,wfo-0000063493
4,Acanthospermum hispidum,None,None,None,None,None,NaN,NaN,hispid,None,...,NaN,NaN,None,None,None,None,None,None,pt,wfo-0000068556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,Alternanthera pungens,None,None,prostrate,None,None,NaN,NaN,present,None,...,NaN,NaN,None,None,None,None,None,None,pt,wfo-0000528787
120,Alternanthera sessilis,None,None,prostrate,None,None,NaN,NaN,absent/present,None,...,NaN,NaN,None,None,None,None,None,None,pt,wfo-0000528825
121,Amaranthus caudatus,None,None,None,None,None,0.3,1.0,None,None,...,NaN,NaN,None,None,None,None,None,None,de,wfo-0000530136
122,Amaranthus viridis,None,None,None,None,None,NaN,NaN,glabrous,absent,...,NaN,NaN,None,None,None,None,None,None,pt,wfo-0000530552


In [24]:
# df.to_csv('wfo-traits-enfr-2.csv', index=False)
df.to_csv('wfo-traits-dept.csv', index=False)

### German and Portuguese

In [19]:
base_dir = Path("scraped_data")
file = base_dir / "merged_results_dedup_lang.csv"
df_dedup = pd.read_csv(file, encoding="latin1")

desc_df = df_dedup[(df_dedup.section.isin(['general', 'morphology'])) & (df_dedup.lang.isin(['de', 'pt']))]

print(f"There are {len(desc_df)} descriptions are in German and Portuguese.")

There are 124 descriptions are in German and Portuguese.


In [85]:
import pandas as pd
import numpy as np
import re

def merge_trait_rows(df: pd.DataFrame) -> pd.DataFrame:
    def merge_group(group: pd.DataFrame):
        merged = {}
        for col in group.columns:
            # Skip NaNs
            col_vals = group[col].dropna()
            
            if col_vals.empty:
                merged[col] = np.nan
                continue

            # Handle numeric columns with min/max logic
            if any(x in col.lower() for x in ["min", "max", "number", "depth", "width", "length", "diameter", "height"]):
                # try to convert to numeric
                col_vals = pd.to_numeric(col_vals, errors="coerce").dropna()
                if col_vals.empty:
                    merged[col] = np.nan
                elif "min" in col.lower():
                    merged[col] = col_vals.min()
                elif "max" in col.lower():
                    merged[col] = col_vals.max()
                else:
                    # for number-like columns without min/max
                    # merged[col] = col_vals.mean() if len(col_vals) > 1 else col_vals.iloc[0]
                    
                    # For number-like columns without min/max — show range instead of mean
                    if len(col_vals) > 1:
                        merged[col] = f"{col_vals.min()}–{col_vals.max()}"
                    else:
                        merged[col] = col_vals.iloc[0]
            else:
                # For text/categorical traits
                # merged[col] = "; ".join(sorted(set(col_vals.astype(str))))
                
                # --- Improved text/categorical merging ---
                tokens = []
                for val in col_vals.astype(str):
                    # Split by "/", ";", "," or " and "
                    parts = re.split(r'[/;,]| and ', val)
                    for p in parts:
                        p = p.strip().lower()
                        if p:
                            tokens.append(p)
                if tokens:
                    merged[col] = "; ".join(sorted(set(tokens)))
                else:
                    merged[col] = np.nan
                    
        return pd.Series(merged)

    # merged_df = df.groupby("name", dropna=False).apply(merge_group).reset_index(drop=True)

    merged_df = (
        df.groupby("name", dropna=False, sort=False)
          .apply(merge_group)
          .reset_index(drop=True)
    )

    return merged_df


In [106]:
df = pd.read_csv("wfo-traits-dept.csv", encoding="utf-8")

In [107]:
def safe_fix_encoding(text):
    """Fix double-encoded mojibake like 'krÃ¤ftig' → 'kräftig' safely."""
    if not isinstance(text, str):
        return text
    try:
        # common case: utf-8 text double-saved as latin1
        return text.encode("latin1").decode("utf-8")
    except (UnicodeEncodeError, UnicodeDecodeError):
        try:
            # fallback: sometimes reversed (latin1 → utf8 → latin1)
            return text.encode("utf-8").decode("latin1")
        except Exception:
            # leave untouched if both fail
            return text

# Apply to all object columns
for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].apply(safe_fix_encoding)

In [108]:
merged_df = merge_trait_rows(df)
print(f"✅ Merged dataframe saved: {len(merged_df)} rows")

✅ Merged dataframe saved: 116 rows


/var/folders/wl/chn7kv1j6zv3b8m_006hckv1kdk2my/T/ipykernel_28361/900031713.py:59: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(merge_group)


In [110]:
merged_df = merged_df.sort_values("name").reset_index(drop=True)

In [111]:
merged_df

,name,life form,habitat,habit,clonality,perennial organ,plant min. height [m],plant max. height [m],indumentum,spinescence,...,seed min. length [mm],seed max. length [mm],seed min. diameter [mm],seed max. diameter [mm],dispersal mode,ploidy level (2n),root min. depth [cm],root max. depth [cm],lang,wfo_taxon_id
0,abrus fruticulosus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,present,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pt,wfo-0000204238
1,acanthospermum hispidum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hispid,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pt,wfo-0000068556
2,achyranthes aspera,NaN,NaN,erect,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pt,wfo-0000516177
3,acmella uliginosa,NaN,NaN,decumbent; erect,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,pappus bristly,NaN,NaN,NaN,pt,wfo-0000132960
4,acroceras zizanioides,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pt,wfo-0000841421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,tribulus terrestris,NaN,NaN,niederliegend; verzweigt,NaN,NaN,0.1,0.6,behaart,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,de,wfo-0000457857
112,trichanthecium parvifolium,annual,NaN,cespitose,NaN,NaN,NaN,NaN,glabrate,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pt,wfo-0001043444
113,vigna luteola,NaN,NaN,prostrate; twining,NaN,NaN,NaN,NaN,hirsute; pilose,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pt,wfo-0000188562
114,vigna unguiculata,NaN,NaN,voluble,NaN,NaN,NaN,NaN,glabrous,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pt,wfo-0000189386


In [112]:
merged_df.to_csv("wfo-traits-merged-dept.csv", index=False)

In [113]:
df = pd.read_csv("wfo-traits-enfr-2.csv", encoding="latin1")

# Apply to all object columns
for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].apply(safe_fix_encoding)

merged_df = merge_trait_rows(df)
print(f"✅ Merged dataframe saved: {len(merged_df)} rows")

✅ Merged dataframe saved: 483 rows


/var/folders/wl/chn7kv1j6zv3b8m_006hckv1kdk2my/T/ipykernel_28361/900031713.py:59: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(merge_group)


In [114]:
merged_df = merged_df.sort_values("name").reset_index(drop=True)

In [115]:
merged_df.to_csv("wfo-traits-merged-enfr-2.csv", index=False)

In [116]:
merged_df

,name,life form,habitat,habit,clonality,perennial organ,plant min. height [m],plant max. height [m],indumentum,spinescence,...,seed min. diameter [mm],seed max. diameter [mm],dispersal mode,ploidy level (2n),root min. depth [cm],root max. depth [cm],lang,wfo_taxon_id,seed min. length [cm],seed max. length [cm]
0,abutilon pannosum,perennial herb to shrub,NaN,erect,NaN,NaN,1.50,3.00,stellate pubescent; velvety,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,en,wfo-0000512163,NaN,NaN
1,acalypha ciliata,annual; annual herb; herb,NaN,erect; slender; unbranched,NaN,NaN,0.00,1.30,crisped-puberulous; glabrous; pilose; puberulo...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,en,wfo-0000808761; wfo-0000809786,NaN,NaN
2,achyranthes aspera,annuals or perennials; generally perennial; he...,waste places,erect; erect or procumbent; more or less prost...,NaN,root more or less lignified; taproot,0.20,6.00,generally covered with dense; glabrous to dens...,spine 3–4 mm; spinescent; spinose; spiny,...,NaN,NaN,NaN,48; 84; 96]; [42,NaN,NaN,en; fr,wfo-0000516177,NaN,NaN
3,acrachne racemosa,annual; caespitose annual; tufted annual,NaN,erect or geniculately ascending; slender erect...,NaN,NaN,0.08,0.80,glabrous; glabrous or sparsely pilose on the u...,NaN,...,NaN,NaN,NaN,36,NaN,NaN,en,wfo-0000841378,NaN,NaN
4,acridocarpus chevalieri,NaN,NaN,climbing shrub,NaN,NaN,NaN,NaN,pubescent,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,en,wfo-0000518510,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,zehneria thwaitesii,climber or trailer; herbaceous liana,NaN,climbing or creeping,monoecious,NaN,1.00,2.00,glabrescent or slightly hairy; glabrous or sli...,NaN,...,0.3,0.5,NaN,NaN,NaN,NaN,en; fr,wfo-0000430217,NaN,NaN
479,ziziphus mucronata,shrub; shrub or small tree; shrub or tree; shr...,NaN,decumbent; erect or scandent,with rhizomes,NaN,1.00,30.00,becoming glabrous; glabrous or pubescent; youn...,armed with spinous stipules; of variable dimen...,...,NaN,NaN,NaN,NaN,NaN,NaN,en; fr,wfo-0000430319,NaN,NaN
480,ziziphus spina-christi,tree,NaN,spreading,NaN,NaN,NaN,NaN,glabrous or slightly pubescent,stipular spines in pairs,...,NaN,NaN,NaN,NaN,NaN,NaN,en,wfo-0001131308,NaN,NaN
481,zygotritonia bongensis,herb,NaN,NaN,NaN,corm,0.18,0.65,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fr,wfo-0000787418,NaN,NaN


In [99]:
merged_df

,name,life form,habitat,habit,clonality,perennial organ,plant min. height [m],plant max. height [m],indumentum,spinescence,...,seed min. diameter [mm],seed max. diameter [mm],dispersal mode,ploidy level (2n),root min. depth [cm],root max. depth [cm],lang,wfo_taxon_id,seed min. length [cm],seed max. length [cm]
0,scleria pergracilis,annual; annuals,NaN,caespitose; erect; slender; tufted,NaN,rhizomes absent,0.1100,0.6000,glabrous,NaN,...,1.0,3.0,NaN,10,NaN,NaN,en,wfo-0000552165,NaN,NaN
1,scleria tessellata,annual,NaN,tufted,NaN,NaN,0.6000,0.9000,glabrous,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,en,wfo-0000553106,NaN,NaN
2,siphonochilus aethiopicus,perennial herb,NaN,NaN,NaN,rhizome,NaN,0.7000,glabrous,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,en,wfo-0000553574,NaN,NaN
3,bacopa crenata,annual herb; annual or perhaps sometimes peren...,NaN,ascending; creeping; creeping to erect; decumb...,NaN,NaN,0.0000,0.9144,glabrous; subglabrous,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,en; fr,wfo-0000558234,NaN,NaN
4,bacopa decumbens,herb,NaN,erect,NaN,NaN,0.3048,0.6096,soft,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,en,wfo-0000558237,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,ammannia auriculata,annual herb; erect; herb; herbs,borders of ponds; similar wet places,erect; erect or decumbent; erect simple or ± b...,NaN,NaN,0.0600,0.8000,sometimes scabrous; sometimes ± scabrous,NaN,...,NaN,NaN,NaN,32; 32]; [30,NaN,NaN,en; fr,wfo-0000530923,NaN,NaN
479,ammannia prieuriana,annual herb; herb,NaN,branched; erect; erect or straggling,NaN,NaN,0.1000,1.2000,NaN,NaN,...,NaN,1.5,NaN,NaN,NaN,NaN,en; fr,wfo-0000531051,NaN,NaN
480,ampelocissus africana,climber; climber or liane; trailer or extensiv...,NaN,prostrate,NaN,rootstock,1.0000,6.0000,glabrous to pubescent or cottony or sometimes ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,en,wfo-0000531453,NaN,NaN
481,ampelocissus leonensis,NaN,NaN,climber,NaN,NaN,NaN,NaN,pubescent,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,en,wfo-0000531536,NaN,NaN


In [117]:
filtered = merged_df[merged_df["name"].str.lower() == "ammannia auriculata"]
filtered

,name,life form,habitat,habit,clonality,perennial organ,plant min. height [m],plant max. height [m],indumentum,spinescence,...,seed min. diameter [mm],seed max. diameter [mm],dispersal mode,ploidy level (2n),root min. depth [cm],root max. depth [cm],lang,wfo_taxon_id,seed min. length [cm],seed max. length [cm]
21,ammannia auriculata,annual herb; erect; herb; herbs,borders of ponds; similar wet places,erect; erect or decumbent; erect simple or ± b...,NaN,NaN,0.06,0.8,sometimes scabrous; sometimes ± scabrous,NaN,...,NaN,NaN,NaN,32; 32]; [30,NaN,NaN,en; fr,wfo-0000530923,NaN,NaN


In [120]:
filtered['plant min. height [m]']

21    0.06
Name: plant min. height [m], dtype: float64

In [101]:
filtered = df[df["name"].str.lower() == "ammannia auriculata"]
print(filtered)

                     name    life form  \
1517  Ammannia auriculata        Herbs   
1518  Ammannia auriculata          NaN   
1519  Ammannia auriculata        Erect   
1520  Ammannia auriculata         herb   
1521  Ammannia auriculata  Annual herb   
1522  Ammannia auriculata  Annual herb   
1523  Ammannia auriculata  Annual herb   
1524  Ammannia auriculata         Herb   

                                       habitat                       habit  \
1517                                       NaN                         NaN   
1518                                       NaN          erect or decumbent   
1519  Borders of ponds, and similar wet places        pyramidally branched   
1520                                       NaN                       erect   
1521                                       NaN  erect simple or ± branched   
1522                                       NaN  erect simple or ± branched   
1523                                       NaN                       Erect 

### Old

In [37]:
data = []
x = 0
for i, row in tqdm(desc_df.iterrows(), total=len(desc_df), desc="Extracting traits"):
    try:
        traits = extract_traits(row.text, row.lang)
    except Exception:
        continue

    traits['name'] = row['name']
    traits['lang'] = row.lang
    traits['wfo_taxon_id'] = row.wfo_taxon_id
    data.append(traits)
    x+=1
    if x > 934:
        break

Extracting traits:  38%|███████▏           | 935/2484 [00:00<00:00, 2916.96it/s]


In [38]:
df = pd.DataFrame(data)

In [39]:
df

,life form,habitat,habit,clonality,perennial organ,plant min. height [m],plant max. height [m],indumentum,spinescence,succulence,...,seed max. length [mm],seed min. diameter [mm],seed max. diameter [mm],dispersal mode,ploidy level (2n),root min. depth [cm],root max. depth [cm],name,lang,wfo_taxon_id
0,Annuals,None,Culms densely tufted,None,None,0.02,0.25,None,None,None,...,NaN,NaN,NaN,None,None,NaN,NaN,Cyperus michelianus,en,wfo-0001302265
1,Tufted annual,None,None,None,None,0.03,0.1,None,None,None,...,NaN,NaN,NaN,None,None,NaN,NaN,Cyperus michelianus,en,wfo-0001302265
2,annual herb,None,None,None,None,0.30,0.9,glabrous,None,None,...,NaN,NaN,NaN,None,None,NaN,NaN,Sphaeranthus talbotii,en,wfo-0000002780
3,herb,None,decumbent or scrambling,often rooting at the nodes,None,0.15,0.6,glabrous or pilose,None,sappy,...,2.4,NaN,NaN,None,None,NaN,NaN,Acmella caulirhiza,en,wfo-0000009712
4,Herbaceous,None,decumbent or almost erect,often rooting,None,0.15,None,glabrous or pilose,None,aqueous stem,...,2.4,NaN,NaN,"pappus generally absent, but sometimes those o...",None,NaN,NaN,Acmella caulirhiza,fr,wfo-0000009712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,Perennial herb,None,"erect, bushy, sprawling, sub-scandent",None,None,0.60,2,thinly pilose to densely tomentose,Spines of modified flowers,None,...,2.0,NaN,NaN,None,None,NaN,NaN,Pupalia lappacea,en,wfo-0000394468
931,None,None,weed,None,None,0.61,1.22,None,None,None,...,NaN,NaN,NaN,None,None,NaN,NaN,Pupalia lappacea,en,wfo-0000394468
932,Annual or perennial herb,None,"more or less erect, prostrate and sprawling, o...",None,None,0.30,2.5,thinly pilose to densely tomentose,None,None,...,2.0,NaN,NaN,None,None,NaN,NaN,Pupalia lappacea,en,wfo-0000394468
933,Annual or perennial herb,None,"erect, prostrate and sprawling, or subscandent...",None,None,0.30,2.5,"sericeous, tomentose, subglabrous, pilose",None,None,...,2.0,NaN,NaN,None,None,NaN,NaN,Pupalia lappacea,en,wfo-0000394468


In [40]:
col = 'name'
df = df[[col] + [c for c in df.columns if c != col]]

In [41]:
df

,name,life form,habitat,habit,clonality,perennial organ,plant min. height [m],plant max. height [m],indumentum,spinescence,...,seed min. length [mm],seed max. length [mm],seed min. diameter [mm],seed max. diameter [mm],dispersal mode,ploidy level (2n),root min. depth [cm],root max. depth [cm],lang,wfo_taxon_id
0,Cyperus michelianus,Annuals,None,Culms densely tufted,None,None,0.02,0.25,None,None,...,NaN,NaN,NaN,NaN,None,None,NaN,NaN,en,wfo-0001302265
1,Cyperus michelianus,Tufted annual,None,None,None,None,0.03,0.1,None,None,...,NaN,NaN,NaN,NaN,None,None,NaN,NaN,en,wfo-0001302265
2,Sphaeranthus talbotii,annual herb,None,None,None,None,0.30,0.9,glabrous,None,...,NaN,NaN,NaN,NaN,None,None,NaN,NaN,en,wfo-0000002780
3,Acmella caulirhiza,herb,None,decumbent or scrambling,often rooting at the nodes,None,0.15,0.6,glabrous or pilose,None,...,1.3,2.4,NaN,NaN,None,None,NaN,NaN,en,wfo-0000009712
4,Acmella caulirhiza,Herbaceous,None,decumbent or almost erect,often rooting,None,0.15,None,glabrous or pilose,None,...,1.3,2.4,NaN,NaN,"pappus generally absent, but sometimes those o...",None,NaN,NaN,fr,wfo-0000009712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,Pupalia lappacea,Perennial herb,None,"erect, bushy, sprawling, sub-scandent",None,None,0.60,2,thinly pilose to densely tomentose,Spines of modified flowers,...,2.0,2.0,NaN,NaN,None,None,NaN,NaN,en,wfo-0000394468
931,Pupalia lappacea,None,None,weed,None,None,0.61,1.22,None,None,...,NaN,NaN,NaN,NaN,None,None,NaN,NaN,en,wfo-0000394468
932,Pupalia lappacea,Annual or perennial herb,None,"more or less erect, prostrate and sprawling, o...",None,None,0.30,2.5,thinly pilose to densely tomentose,None,...,2.0,2.0,NaN,NaN,None,None,NaN,NaN,en,wfo-0000394468
933,Pupalia lappacea,Annual or perennial herb,None,"erect, prostrate and sprawling, or subscandent...",None,None,0.30,2.5,"sericeous, tomentose, subglabrous, pilose",None,...,2.0,2.0,NaN,NaN,None,None,NaN,NaN,en,wfo-0000394468


In [42]:
df.to_csv('wfo-traits-enfr.csv', index=False)

In [ ]:
df.wfo_taxon_id